## <font color='blue'>Utilisation d'estimateurs Scala avec Pyspark via un wrapper </font>

L'objectif est de montrer avec un exemple simple commment créer ses propres estimators et transformers.

1. L'estimator ou le transformer est d'abord créer en scala 
2. Ensuite on crée un wrapper python pour l'utiliser avec Pyspark

Cette méthode permet aux estimators et aux transformers de fonctionnalités de Pyspark.

Créer des estimators avec Pyspark directement peut poser des problèmes notemment lors de la sauvegarde et lecture des modèles.

Le code est [ici](python)

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('Exple')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

from feature import ColumnFilter, CustomVectorIndexer, CustomVectorIndexerModel

In [3]:
# créer une dataFrame à partir d’une séquence

seq =  [(0, "aa"), (1, "ab"), (2, "cd"), (3, "ad"), (4, "aa")]
dfm = spark.createDataFrame(seq).toDF("ID", "CATEGORY")
dfm.show(5)

+---+--------+
| ID|CATEGORY|
+---+--------+
|  0|      aa|
|  1|      ab|
|  2|      cd|
|  3|      ad|
|  4|      aa|
+---+--------+



**Transformer**

In [4]:
filter_ = ColumnFilter().setDropCol('ID')
filter_.transform(dfm).show()

+--------+
|CATEGORY|
+--------+
|      aa|
|      ab|
|      cd|
|      ad|
|      aa|
+--------+



**Estimator**

In [5]:
vectorizer = CustomVectorIndexer().setInputCol('CATEGORY').setOutputCol('CATEGORY_INDEX')
model  = vectorizer.fit(dfm)
model.transform(dfm).show(truncate=False)

+---+--------+--------------+
|ID |CATEGORY|CATEGORY_INDEX|
+---+--------+--------------+
|0  |aa      |0.0           |
|1  |ab      |1.0           |
|2  |cd      |3.0           |
|3  |ad      |2.0           |
|4  |aa      |0.0           |
+---+--------+--------------+



In [6]:
model.mapping_cat

{'aa': 0, 'ab': 1, 'ad': 2, 'cd': 3}